# Extended Sources

So far we have considered earthquakes as point source of seismic energy.  This is clearly incorrect, and for moderate to
large magnitude earthquakes, the difference between point sources and the true earthquake source can be resolvable with
seismic data.  When we considered earthquake location, we were thinking about the earthquakes hypocentre: where the
earthquake started.  Another commonly used "point on a map" type location is the centroid location: the centre of an
earthquakes energy release.

In this notebook we will consider the effects of an extended source on the observed seismograms.  We will briefly cover
the ideas behind:
- Moment-rate functions
- Centroid moment tensors
- Source models (Brune etc.)
- Directivity

# Get them to read one or more Kaikoura paper on the source model. Maybe compare the geodetic results to the teleseismic - think about resolution.